# Differences between Rotten Tomatoes' Tomatometer and Audience score - filtered by genre.
Does the difference between critic score and audience score vary by genre. The project's idea is to mine the famous [Rotten Tomatoes](https://www.rottentomatoes.com) for the scores of movies and then look-up the genres in imdb.

Once I got the data I will store it in a MySQL database and query from there.

In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [2]:
# Defining the options for our browser
chrome_options = Options()
chrome_options.add_argument("--headless") # to not display the page

# Defining the drivers and the URL
driver = webdriver.Chrome(chrome_options=chrome_options)
driver.get('https://www.rottentomatoes.com/browse/dvd-streaming-all/')
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [3]:
# Scraping Functions
def get_movie_count(webdriver):
    html = BeautifulSoup(webdriver.page_source, 'html.parser')
    init_movie_count = html.find(attrs={"id": "count-link"}).findAll('span')[2].text.split()[1]
    movie_count = html.find(attrs={"id": "count-link"}).findAll('span')[2].text.split()[-1]
    
    return int(init_movie_count), int(movie_count)

def click_on_show_more(webdriver):
    webdriver.find_element_by_css_selector('.mb-load-btn').click()
    # no returns

In [4]:
get_movie_count(driver)

(32, 19636)

In [5]:
init_count, total_count = get_movie_count(driver)

In [6]:
while init_count < total_count:
    click_on_show_more(driver)
    init_count, total_count = get_movie_count(driver)

In [7]:
get_movie_count(driver)

(9984, 0)

In [20]:
def get_all_movies(webdriver):
    html = BeautifulSoup(webdriver.page_source, 'html.parser')
    movie_infos = html.find_all(attrs={'class':'movie_info'})
    
    item = 0
    for info in movie_infos:
        print(info.find(attrs={'class':'movieTitle'}).text)
        print(info.find_all(attrs={'class':'tMeterScore'}))
        item += 1
        break

In [21]:
movie_info = get_all_movies(driver)

Avengers: Infinity War
[<span class="tMeterScore"><!-- react-text: 240 -->83<!-- /react-text --><!-- react-text: 241 -->%<!-- /react-text --></span>, <span class="tMeterScore"><!-- react-text: 246 -->91<!-- /react-text --><!-- react-text: 247 -->%<!-- /react-text --></span>]


In [13]:
type(movie_info)

NoneType